# PERBANDINGAN 1 : Feedback Data Analysis with Imbalance Data

1. Specification : 2 class (positive and negative)
2. Data Preprocessing : Data Augmentation  
3. Model : SVM




## Library

In [ ]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from google.colab import drive #akses data dari google drive

In [ ]:
#filtering
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Dataset

In [ ]:
drive.mount('/content/gdrive')
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/dataset_unbalanced.csv')
df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Comment,VALUE
0,alat kebersihan yang ada di asrama putri kuran...,negatif
1,untuk fasilitas olahraga kurang sepakbola,negatif
2,kurang lengkap untuk alat praktek,negatif
3,Saran saya agar bdi bisa memperbaiki fasilitas...,negatif
4,untuk fasilitas olahraga kurang sepakbola,negatif


In [ ]:
#rename kolom
df = df.rename(columns={'Comment': 'komen','VALUE': 'kategori'})
df.head(200)

,komen,kategori
0,alat kebersihan yang ada di asrama putri kuran...,negatif
1,untuk fasilitas olahraga kurang sepakbola,negatif
2,kurang lengkap untuk alat praktek,negatif
3,Saran saya agar bdi bisa memperbaiki fasilitas...,negatif
4,untuk fasilitas olahraga kurang sepakbola,negatif
...,...,...
195,BDI pokokya mantap,positif
196,Sangat jelas,positif
197,baguss dan cocok,positif
198,Sudah ok,positif


In [ ]:
df.isnull().sum() #cek dataset apakah ada yang kosong

komen       0
kategori    0
dtype: int64

# Data Pre-processing - Casefolding

In [ ]:
#casefolding preprocessing
import re
def casefolding(komen):
	komen = komen.lower()
	komen = komen.strip(" ")
	komen = re.sub(r'[?|$|.|&,-/]','',komen)
	return komen
df['komen'] = df['komen'].apply(casefolding)
#type(df.komen[1])

In [ ]:
df.komen[44].split() #mengecek bentuk kalimat

['mohon', 'segera', 'memperbaiki', 'tempat', 'yang', 'rusak']

# Data Pre-processing

Studi kasus teks, augmentasi data bisa dilakukan dengan melakukan tokenizing data menjadi kalimat, mengacak dan menggabungkannya kembali untuk menghasilkan teks baru, atau mengganti kata sifat, kata kerja, dll dengan sinonimnya untuk menghasilkan teks yang berbeda dengan arti yang sama.

# **Tokenization**

Bentuk kalimat saat ini belum berbentuk vector sehingga diperlukan tokenization.Tokenization adalah proses mengubah teks menjadi token sebelum ditransformasi ke vector. 

In [ ]:
#proses tokenizing (pemenggalan per kata)
def token(komentar):
	nstr = komentar.split(' ')
	dat = []
	a = -1
	for h in nstr:
		a+=1
	if h == '':
		dat.append(a)
	p=0
	q=0
	for r in dat:
		q=r-p
		del nstr[q]
		p+=1
	return nstr
df['komen'] = df['komen'].apply(token)
df.head(5)

,komen,kategori
0,"[alat, kebersihan, yang, ada, di, asrama, putr...",negatif
1,"[untuk, fasilitas, olahraga, kurang, sepakbola]",negatif
2,"[kurang, lengkap, untuk, alat, praktek]",negatif
3,"[saran, saya, agar, bdi, bisa, memperbaiki, fa...",negatif
4,"[untuk, fasilitas, olahraga, kurang, sepakbola]",negatif


Stop words digunakan agar kata-kata yang paling sering muncul dan tidak relevan dalam konteks data tidak memberikan kontribusi makna yang lebih dalam pada frasa.

In [ ]:
def stopwords_removal(comments):
	filtering = stopwords.words('indonesian','english')
	x=[]
	data=[]
	def myFunc(x):
		if x in filtering:
			return False
		else:
			return True
	fit = filter(myFunc,comments)
	for x in fit:
		data.append(x)
	return data
df['komen'] = df['komen'].apply(stopwords_removal)
df.head(5)

,komen,kategori
0,"[alat, kebersihan, asrama, putri, lengkap, kai...",negatif
1,"[fasilitas, olahraga, sepakbola]",negatif
2,"[lengkap, alat, praktek]",negatif
3,"[saran, bdi, memperbaiki, fasilitas, kamar, ]",negatif
4,"[fasilitas, olahraga, sepakbola]",negatif


Filtering merupakan proses mengambil kata kata yang utama, menghilangkan kata sambung seperti ke-, yang, dan, atau, dll

In [ ]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
def remove_punctuation(comments):
  txt_nopunct = [c for c in comments if c not in string.punctuation]
  return txt_nopunct

In [ ]:
df['komen'] = df['komen'].apply(lambda x: remove_punctuation(x))
df.head(3)

,komen,kategori
0,"[alat, kebersihan, asrama, putri, lengkap, kai...",negatif
1,"[fasilitas, olahraga, sepakbola]",negatif
2,"[lengkap, alat, praktek]",negatif


In [ ]:
pip install Sastrawi

     |████████████████████████████████| 209 kB 5.3 MB/s 


**Stemming** bertujuan untuk menghilangkan imbuhan kata dan merubah menjadi kata dasar dengan bantuan library Sastrawi

In [ ]:
#stemming
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(comments):
	factory = StemmerFactory()
	stemmer = factory.create_stemmer()
	do = []
	for w in comments:
		dt = stemmer.stem(w)
		do.append(dt)
	d_clean=[]
	d_clean=" ".join(do)
#	print(d_clean)
	return d_clean
df['komen'] = df['komen'].apply(stemming)

In [ ]:
#df.to_csv('gdrive/My Drive/Colab Notebooks/dataset_clean.csv',index=False)
df_clean = pd.read_csv('gdrive/My Drive/Colab Notebooks/dataset_clean.csv',encoding='latin1')
df_clean.head()

,Comment,VALUE
0,alat kebersihan yang ada di asrama putri kuran...,negatif
1,untuk fasilitas olahraga kurang sepakbola,negatif
2,kurang lengkap untuk alat praktek,negatif
3,Saran saya agar bdi bisa memperbaiki fasilitas...,negatif
4,untuk fasilitas olahraga kurang sepakbola,negatif


In [ ]:
#rename kolom
df_clean = df_clean.rename(columns={'Comment': 'komen','VALUE': 'kategori'})
df_clean.komen[44]

'mohon segera memperbaiki tempat- yang rusak'

In [ ]:
df_clean = df_clean.astype({'kategori':'category'})
df_clean = df_clean.astype({'komen':'string'})
df_clean.dtypes

komen         string
kategori    category
dtype: object

Ekstrasi fitur dari dataset yang sudah bersih menggunakan fitur TF-IDF, yakni pembobotan kata berdasarkan keseluruhan text yang ada di dataset

In [ ]:
#TF-IDF dan 
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(df_clean['komen'].astype('U'))
#type(text_tf)
text_tf.get_shape()

(433, 562)

# Classification - SVM

Linear SVM

In [ ]:
#data split 80:20
#classification
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot

X_train,X_test,y_train,y_test = train_test_split(text_tf,df_clean['kategori'],test_size=0.2,random_state=43)

#SVM
#config model
clf = svm.LinearSVC(class_weight='balanced')
#fit model di dataset pelatihan
clf.fit(X_train,y_train)
#evaluasi dataset pelatihan
predicted = clf.predict(X_test)
#cetak hasil prediksi
print("Akurasi: ",accuracy_score(y_test,predicted))
print("Presisi: ",precision_score(y_test,predicted,average="binary",pos_label="negatif"))
print("Recall: ",recall_score(y_test,predicted,average="binary",pos_label="negatif"))
print("F1 Score:",f1_score(y_test,predicted,average="binary",pos_label="negatif"))

print(f'confusion matrix:\n {confusion_matrix(y_test,predicted)}')
print('===================================================\n')
print(classification_report(y_test,predicted,zero_division=0))

Akurasi:  0.9655172413793104
Presisi:  1.0
Recall:  0.9
F1 Score: 0.9473684210526316
confusion matrix:
 [[27  3]
 [ 0 57]]

              precision    recall  f1-score   support

     negatif       1.00      0.90      0.95        30
     positif       0.95      1.00      0.97        57

    accuracy                           0.97        87
   macro avg       0.97      0.95      0.96        87
weighted avg       0.97      0.97      0.97        87



In [ ]:
cnf_matrix = confusion_matrix(y_test,predicted)
FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# sensitivitas atau true positive rate
TPR = TP/(TP+FN)
# spesifisitas atau true negative rate
TNR = TN/(FP+TN) 
# presisi 
PPV = TP/(TP+FP)
# false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
#error
ERROR = (FP+FN)/(TP+FP+FN+TN)

In [ ]:
# sensitivitas atau true positive rate
print('Sensitivitas : ', TPR)
# spesifisitas atau true negative rate
print('Spesifisitas :', TNR)
# presisi 
print('Presisi : ', PPV)
# false positive rate
print('False Positive Rate : ',FPR)
# False negative rate
print('False Negative Rate : ',FNR)
#error
print('Error : ', ERROR)

Sensitivitas :  [0.9 1. ]
Spesifisitas : [1.  0.9]
Presisi :  [1.   0.95]
False Positive Rate :  [0.  0.1]
False Negative Rate :  [0.1 0. ]
Error :  [0.03448276 0.03448276]
